In [1]:
import genjax
from genjax import dippl
from genjax import gensp
import jax
import adevjax

console = genjax.pretty(show_locals=True)
key = jax.random.PRNGKey(314159)

I0000 00:00:1696004302.286265  103625 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
@genjax.gen
def model(p):
    v = dippl.flip_enum(p) @ "v"
    return v


lifted = gensp.choice_map_distribution(model, genjax.select("v"), None)


def emitter(key, p):
    key, sub_key = jax.random.split(key)
    key, logpdf_key = jax.random.split(key)

    @adevjax.adev
    def flip_loss(p):
        (_, v) = lifted.random_weighted(sub_key, 0.3)
        w = lifted.estimate_logpdf(logpdf_key, v, p)
        b = v.get_leaf_value()["v"]
        return jax.lax.cond(
            b,
            lambda _: w * 0.0,
            lambda p: w * (-p / 2.0),
            p,
        )

    return adevjax.E(flip_loss).grad_estimate(key, p)


jax.jit(emitter)(key, (0.3,))

Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>


(Array(0.27483624, dtype=float32),)